In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
# df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
# df = pd.concat([df_jan, df_feb], axis=0, ignore_index=True)

In [4]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [25]:
#### QUESTION 2 ####

In [23]:
res = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
# print(res.isna().sum())
# df_jan[res.isna()].sample() #[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].sample()
(res.dt.total_seconds()/60).std()

42.594351241920904

In [ ]:
raise Exception("sd")